In [10]:
# Simple interactive Python debugger





# Our buggy program
def remove_html_markup(s):
    tag   = False
    quote = False
    out   = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c
    return out
    
# main program that runs the buggy program
def main():
    print remove_html_markup('"<b>foo</b>"')

    
# ------------ my-spyder --------------------------
""" *** INSTRUCTIONS ***
1. Improve and expand the 'debug' function to accept a print command 'p <arg>'.
If the print command has no argument, print out the dictionary that 
holds all variables.
Print the value of the supplied variable in a form 'var = repr(value)', 
if the 'p' is followed by an argument, or print 'No such variable:', arg 
if no such variable exists.

2. Improve and expand the 'debug' function to accept a breakpoint 
command 'b <line>'.
Add the line number to the breakpoints dictionary, or print 
'You must supply a line number' if 'b' is not followed by a line number.

3. Improve and expand the 'debug' function to accept a watchpoint 
command 'w <var name>'.
Add the variable name to the watchpoints dictionary, or print 
'You must supply a variable name' if 'w' is not followed by a string.

4. Improve and expand the debug function to accept a new command:
a delete command 'd <type> <argument>', where <type> is either b for breakpoint,
or w for watchpoint. The following argument should be either a number 
for the breakpoint or a string for the watchpoint. 
If there is mismatch between type and argument, you should print out
"Incorrect command".
In the case of "d b <argument>" you should delete that breakpoint from the 
breakpoint dictionary, or print "No such breakpoint defined", repr(argument)
In case of watchpoint, you should delete the watchpoint if such variable exists,
or print: variable, "is not defined as watchpoint"

5. Improve the traceit function to watch for variables in the watchpoint 
dictionary and print out (literally like that): 
event, frame.f_lineno, frame.f_code.co_name
and then the values of the variables, each in new line, in a format:
somevar ":", "Initialized"), "=>", repr(somevalue)
if the value was not set, and got set in the line, or
somevar ":", repr(old-value), "=>", repr(new-value)
when the value of the variable has changed.
If the value is unchanged, do not print anything.
"""

import sys
import copy


def debug(command, my_locals):
    global stepping
    global breakpoints
    global watchpoints
    
    if command.find(' ') > 0:
        arg = command.split(' ')[1]
    else:
        arg = None

    if command.startswith('s'):       # step
        stepping = True
        return True
    
    elif command.startswith('c'):     # continue
        stepping = False
        return True
    
    elif command.startswith('p'):     # print 
        if arg is None:
            print my_locals
        else:
            if arg in my_locals:
                print "{} = {}".format(arg, repr(my_locals[arg]))
            else:
                print "No such variable: {}".format(arg)
                return False
        return True
    
    elif command.startswith('b'):     # breakpoint  
        if arg is None:
            print "You must supply a line number"
        else:
            breakpoints[int(arg)] = True
            return True
    
    elif command.startswith('w'):     # watch variable
        if arg is None:
            print "You must supply a variable name"
        else:
            # We need to use copy.deepcopy to take into account mutable objects that 
            # might change during the execution.
            if arg in my_locals:
                watchpoints[arg] = copy.deepcopy(my_locals[arg])
            else:
                watchpoints[arg] = "variable_not_set"
    
    elif command.startswith('d'):    # delete watch/break point
        if arg is None:
            print """Arguments missing. Correct usage: d <type> <argument>, <type> can be 
            either 'b' for breakpoint or 'w' for watchpoint, <argument> should be a
            number for breakpoints or a string for watchpoints."""
        arg = command.split(' ')[1:]
        if len(arg) != 2:
            print "Delete command takes 2 arguments, {} given.".format(len(arg))
        elif arg[0] not in ['b', 'w']:
            print "Incorrect command."
        elif arg[0] == 'b':     # delete breakpoint
            try:
                line = int(arg[1])
            except:
                print "Incorrect command."
                return False
            if line not in breakpoints: 
                print "No such breakpoint defined", repr(line)
            else:
                del breakpoints[line]
                return True
        else:                   # delete watchpoint
            if arg[1] not in watchpoints: 
                print "{}, is not defined as watchpoint".format(arg[1])
            else:
                del watchpoints[arg[1]]
                return True
            
    elif command.startswith('q'):     # quit
        print "Exiting my-spyder..."
        sys.exit(0)
    
    else:
        print "No such command", repr(command)  
    
    return False
  

def input_command():
    # command = raw_input("(my-spyder): ")
    global commands
    command = commands.pop(0)
    return command


def traceit(frame, event, trace_arg):
    """The local trace function should return a reference to itself 
    (or to another function for further tracing in that scope), 
    or None to turn off tracing in that scope."""
    global stepping
    global breakpoints
    global watchpoints
    global previous_line

    if event == 'line':
        message = "{}, {}, {}\n".format(event, previous_line, frame.f_code.co_name)
        changes = ""
        for variable, value in watchpoints.items():
            if variable in frame.f_locals:
                current_value = copy.deepcopy(frame.f_locals[variable])
                if value == "variable_not_set":
                    changes += "{} : Initialized => {} \n".format(variable, repr(current_value))
                    watchpoints[variable] = current_value
                elif current_value != value:
                    changes += "{} : {} => {} \n".format(variable, repr(value), repr(current_value))
                    watchpoints[variable] = current_value
            else:
                watchpoints[variable] = "variable_not_set"
        if changes:
            print message, changes
            resume = False
            while not resume:
                command = input_command()
                resume = debug(command, frame.f_locals)
        
        if stepping or frame.f_lineno in breakpoints:
            print event, frame.f_lineno, frame.f_code.co_name, frame.f_locals
            resume = False
            while not resume:
                command = input_command()
                resume = debug(command, frame.f_locals)
    
        previous_line = frame.f_lineno
    
    return traceit


# globals
breakpoints = {}
watchpoints = {"quote": True}
watch_values = {}
stepping = True
previous_line = None

commands = ["w c", "c", "c", "w out", "c", "c", "c", "q"]

# Using the tracer
sys.settrace(traceit)
main()
sys.settrace(None)

line 133 __call__ {'source': <_ast.Module object at 0x0000000004152E10>, 'symbol': 'exec', 'self': <IPython.core.compilerop.CachingCompiler instance at 0x0000000003E0F288>, 'filename': '<ipython-input-10-d1b31a53370e>'}
line, 10, remove_html_markup
quote : Initialized => False 

line, 13, remove_html_markup
c : Initialized => '"' 

line, 19, remove_html_markup
quote : False => True 

line, 13, remove_html_markup
c : '"' => '<' 

line, 21, remove_html_markup
out : '' => '<' 

Exiting my-spyder...


SystemExit: 0